# **DEPENDENCIES**

In [ ]:
!pip install rembg numpy pillow opencv-python-headless matplotlib scikit-image scikit-fuzzy onnxruntime joblib keras.models

import joblib
import rembg
import numpy as np
from PIL import Image
import cv2
import matplotlib.pyplot as plt
import matplotlib
from skimage.feature import graycomatrix, graycoprops
from skfuzzy import control as ctrl
import skfuzzy as fuzz
import warnings
import pandas as pd
from keras.models import load_model
import pprint
from google.colab import drive
drive.mount('/content/drive')


# LEAF TYPE PREDICTION

In [ ]:
disease_class_indices_map = {
    "apple": {
        0: "Alternia leaf spot",
        1: "Brown Spot",
        2: "Fray spot",
        3: "Healthy",
        4:"Rust"
    },
    "cotton": {
        0: "bacterial blight",
        1: "curl virus",
        2: "fussarium wilt",
        3: "healthy",

    },
    "grape": {
        0: "Grape___Black_rot",
        1: "Grape___Esca_(Black_Measles)",
        2: "Healthy",
        3: "Leaf Blight"
    },
    "mango": {
        0: "Mango___Anthracnose",
        1: "Mango___Bacterial_Canker",
        2: "Mango___Cutting Weevil",
        3: "Mango___Dieblack",
        4: "Mango___Gall Midge",
        5: "Mango___healthy",
        6: "Mango___Powdery Mildew",
        7: "Mango___Sooty Mould"
    },
    "potato": {
        0: "Bacteria",
        1: "Fungi",
        2: "Healthy",
        3: "Nematode",
        4: "Pest",
        5: "Phytophthora",
        6: "Virus"
    },
    "tomato": {
        0: "Tomato___Bacterial_spot",
        1: "Tomato___Early_blight",
        2: "Tomato___Late_blight",
        3: "Tomato___Leaf_Mold",
        4: "Tomato___Septoria_leaf_spot",
        5: "Tomato___Spider_mites",
        6: "Tomato___Target_Spot",
        7: "Tomato___Tomato_Yellow_Leaf_Curl_Virus",
        8: "Tomato___Tomato_mosaic_virus",
        9: "Tomato___healthy"
    }
}


import numpy as np
from PIL import Image
from tensorflow.keras.models import load_model

# Load and preprocess image
def load_and_preprocess_image(image_path, target_size=(224, 224)):
    img = Image.open(image_path).convert("RGB")
    img = img.resize(target_size)
    img_array = np.array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array.astype('float32') / 255.
    return img_array

# Predict using any model
def predict_image_class(model, image_path, class_indices):
    preprocessed_img = load_and_preprocess_image(image_path)
    predictions = model.predict(preprocessed_img)
    predicted_class_index = np.argmax(predictions, axis=1)[0]
    predicted_class_name = class_indices[predicted_class_index]
    return predicted_class_name

# Example: Predict full pipeline
def predict_leaf_and_disease(image_path):
    # Step 1: Predict leaf type
    print("🔍 Predicting leaf type...")
    leaf_type_model = load_model("/content/drive/MyDrive/leaf project/models/leaf_type.h5")
    leaf_type_class_indices = {0: "apple", 1: "cotton", 2: "grape", 3: "mango", 4: "potato", 5: "tomato"}  # Adjust as per your model
    leaf_type = predict_image_class(leaf_type_model, image_path, leaf_type_class_indices)
    print(f"🌿 Leaf Type Predicted: {leaf_type}")

    # Step 2: Load and predict using specific leaf model
    specific_model_path = f"/content/drive/MyDrive/leaf project/models/model_leaf_{leaf_type}.h5"
    print(f"🔬 Predicting disease using: {specific_model_path}")
    disease_model = load_model(specific_model_path)

    # Define or load specific class_indices for disease model
    # Example for tomato — you should customize this dictionary for each model


    disease_class_indices = disease_class_indices_map.get(leaf_type)
    if not disease_class_indices:
        return f"No class index map found for {leaf_type} leaf type"

    disease = predict_image_class(disease_model, image_path, disease_class_indices)
    print(f"🧪 Disease Prediction: {disease}")

    return leaf_type, disease



input_image_path = '/content/h358.jpg'
img = Image.open(input_image_path).resize((512, 512), Image.LANCZOS)
img.save(input_image_path)

leaf, disease = predict_leaf_and_disease(input_image_path)
print(f"\n✅ Final Result:\nLeaf Type: {leaf}\nPredicted Disease: {disease}")


img = Image.open(input_image_path)

plt.figure(figsize=(6, 6))
plt.imshow(img)
plt.title(f"Predicted disease: {leaf}_{disease}", fontsize=14)
plt.axis('off')
plt.show()

# **BACKGROUND REMOVAL OF INPUT IMAGE**

In [ ]:
def remove_background(image_path):
    input_image = Image.open(image_path)
    input_array = np.array(input_image)
    output_array = rembg.remove(input_array)
    output_image = Image.fromarray(output_array)
    return input_image, output_image

**CHANGE INPUT IMAGE HERE**

In [ ]:
input_image, output_image = remove_background(input_image_path)

plt.subplot(1, 2, 1)
plt.title("Original Image")
plt.imshow(input_image)
plt.axis("off")

plt.subplot(1, 2, 2)
plt.title("Background Removed")
plt.imshow(output_image)
plt.axis("off")

plt.tight_layout()
plt.show()

input_image -> INPUT IMAGE

output_image -> BACKGROUND REMOVED IMAGE

# **GET MANUAL COLOR RANGES FOR EACH LEAF TYPE**

In [ ]:
def get_hsvcolor_ranges(leaf_type):
    if leaf_type == 'potato':
        return {
            'green': (np.array([34, 21, 44]), np.array([78, 255, 255])),
            'yellow': (np.array([20, 50, 50]), np.array([30, 255, 255])),
            'brown': (np.array([10, 100, 20]), np.array([20, 255, 200])),
            'orange': (np.array([10, 50, 50]), np.array([20, 255, 255])),
            'skin': (np.array([0, 48, 80]), np.array([20, 255, 255]))

        }
    elif leaf_type == 'mango':
        return {
            'green': (np.array([28, 44, 39]), np.array([75, 255, 255])),
            'yellow': (np.array([22, 120, 100]), np.array([32, 255, 255])),
            'brown': (np.array([8, 100, 40]), np.array([20, 255, 150])),
            'orange': (np.array([10, 50, 50]), np.array([20, 255, 255])),
            'skin': (np.array([0, 48, 80]), np.array([20, 255, 255])),
            'gray': (np.array([0, 0, 50]), np.array([180, 50, 200])),
            'red': (np.array([0, 50, 50]), np.array([10, 255, 255]))
        }
    elif leaf_type == 'apple':
        return {
            'green': (np.array([41, 24, 24]), np.array([94, 255, 255])),
            'yellow': (np.array([22, 120, 100]), np.array([32, 255, 255])),
            'brown': (np.array([8, 100, 40]), np.array([20, 255, 150])),
            'orange': (np.array([10, 50, 50]), np.array([20, 255, 255])),
            'skin': (np.array([0, 48, 80]), np.array([20, 255, 255])),
            'gray': (np.array([0, 0, 50]), np.array([180, 50, 200])),
            'red': (np.array([0, 50, 50]), np.array([10, 255, 255]))
        }
    else:
        return {
            'green': (np.array([34, 21, 44]), np.array([78, 255, 255])),
            'yellow': (np.array([20, 50, 50]), np.array([30, 255, 255])),
            'brown': (np.array([10, 100, 20]), np.array([20, 255, 200])),
            'orange': (np.array([10, 50, 50]), np.array([20, 255, 255])),
            'skin': (np.array([0, 48, 80]), np.array([20, 255, 255]))

        }

In [ ]:
leaf_type=leaf
color_ranges = get_hsvcolor_ranges(leaf_type)

leaf_type -> LEAF TYPE

color_ranges -> MANUAL HSV COLOR RANGES ACC TO LEAF TYPE

# **MAKE THE MASKS AND GET PERCENTAGES AND GLCM FEATURES**

In [ ]:
def analyze_leaf(image, color_ranges):
    image_bgr = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
    hsv = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2HSV)

    # Green mask
    l_green, u_green = color_ranges['green']
    mask_green = cv2.inRange(hsv, l_green, u_green)

    # Select only available disease masks
    masks = [
        cv2.inRange(hsv, color_ranges[key][0], color_ranges[key][1])
        for key in ['yellow', 'brown', 'orange', 'skin', 'gray', 'red'] if key in color_ranges
    ]

    # Combine available masks if any
    if masks:
        mask_combined = masks[0]
        for mask in masks[1:]:
            mask_combined = cv2.bitwise_or(mask_combined, mask)

        # Apply morphological operations
        kernel = np.ones((3, 3), np.uint8)
        mask_combined = cv2.dilate(mask_combined, kernel, iterations=1)
        mask_combined = cv2.morphologyEx(mask_combined, cv2.MORPH_CLOSE, kernel)
    else:
        mask_combined = np.zeros_like(mask_green)  # No disease pixels

    # Ensure no overlap between green and disease areas
    mask_green = cv2.bitwise_and(mask_green, cv2.bitwise_not(mask_combined))
    # Calculate disease percentage
    total_pixels = image_bgr.shape[0] * image_bgr.shape[1]
    green_pixels = np.sum(mask_green == 255)
    disease_pixels = np.sum(mask_combined == 255)
    disease_percentage = (disease_pixels / (disease_pixels + green_pixels)) * 100
    healthy_percentage = 100 - disease_percentage
    print("Healthy % by color detection",healthy_percentage)
    print("Disease % by color detection",disease_percentage)


    # Extract GLCM features for the diseased area
    gray_image = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2GRAY)
    diseased_region = cv2.bitwise_and(gray_image, gray_image, mask=mask_combined)

    # Compute GLCM
    glcm = graycomatrix(diseased_region, distances=[1], angles=[0], levels=256, symmetric=True, normed=True)

    # Extract all GLCM properties
    contrast = graycoprops(glcm, 'contrast')[0, 0]
    correlation = graycoprops(glcm, 'correlation')[0, 0]
    energy = graycoprops(glcm, 'energy')[0, 0]
    homogeneity = graycoprops(glcm, 'homogeneity')[0, 0]
    asm = graycoprops(glcm, 'ASM')[0, 0]
    dissimilarity = graycoprops(glcm, 'dissimilarity')[0, 0]

    # Print all GLCM features
    print(f"\nGLCM Features:")
    print(f"\nContrast: {contrast:.2f}")
    print(f"Correlation: {correlation:.2f}")
    print(f"Energy: {energy:.2f}")
    print(f"Homogeneity: {homogeneity:.2f}")
    print(f"ASM (Angular Second Moment): {asm:.2f}")
    print(f"Dissimilarity: {dissimilarity:.2f}")


    return mask_green, mask_combined, disease_percentage, contrast, correlation, energy, homogeneity, asm, dissimilarity


In [ ]:
mask_green, mask_combined, disease_percentage, contrast, correlation, energy, homogeneity, asm, dissimilarity = analyze_leaf(
        output_image, color_ranges)


# Apply masks to the output image to visualize regions
output_array = np.array(output_image)

# Green area in color
green_area = cv2.bitwise_and(output_array, output_array, mask=mask_green)

# Diseased area in color
diseased_area = cv2.bitwise_and(output_array, output_array, mask=mask_combined)

# Display both masks applied to the output image
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.title("Green Area")
plt.imshow(green_area)
plt.axis("off")

plt.subplot(1, 2, 2)
plt.title("Diseased Area")
plt.imshow(diseased_area)
plt.axis("off")

plt.tight_layout()
plt.show()


In [ ]:
def visualize_results(input_image, output_image, green_mask, disease_mask, leaf_type, disease_percentage):
    green_overlay = cv2.cvtColor(green_mask, cv2.COLOR_GRAY2BGR)
    disease_overlay = cv2.cvtColor(disease_mask, cv2.COLOR_GRAY2BGR)
    green_overlay[np.where((green_overlay == [255, 255, 255]).all(axis=2))] = [0, 255, 0]
    disease_overlay[np.where((disease_overlay == [255, 255, 255]).all(axis=2))] = [0, 0, 255]

    output_image_bgr = cv2.cvtColor(np.array(output_image), cv2.COLOR_RGB2BGR)
    combined_overlay = cv2.addWeighted(output_image_bgr, 1.0, disease_overlay, 0.3, 0)
    combined_overlay_rgb = cv2.cvtColor(combined_overlay, cv2.COLOR_BGR2RGB)

    fig, axes = plt.subplots(1, 3, figsize=(12, 6))
    axes[0].imshow(input_image)
    axes[0].set_title("Original Image")
    axes[1].imshow(output_image)
    axes[1].set_title("Background Removed")
    axes[2].imshow(combined_overlay_rgb)
    axes[2].set_title("Disease Overlay")
    for ax in axes:
        ax.axis("off")
    fig.text(0.5, 0.02, f"Leaf: {leaf_type}, Disease: {disease_percentage:.2f}%",
             ha="center", fontsize=12, color="red", bbox={"facecolor": "white", "alpha": 0.8, "pad": 5})
    plt.tight_layout(rect=[0, 0.05, 1, 1])
    plt.show()

In [ ]:
visualize_results(input_image, output_image, mask_green, mask_combined, leaf_type, disease_percentage)

mask_green -> GREEN MASK

mask_combined -> DISEASE MASK

disease_percentage -> COLOR DETECTION DISEASE %

contrast, correlation, energy, homogeneity, asm, dissimilarity -> GLCM

# **PREDICT PERCENTAGES USING GLCM FEATURES**

In [ ]:
def predict_disease_glcm(contrast, correlation, energy, homogeneity, asm, dissimilarity, mask_disease_percent, model_path="/content/drive/MyDrive/leaf project/glcm_regressor.pkl"):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")  # Suppress all warnings
        model = joblib.load(model_path)
        features = np.array([[contrast, correlation, energy, homogeneity, asm, dissimilarity]])
        predicted_disease_percent = model.predict(features)[0]

    predicted_healthy_percent = 100 - predicted_disease_percent
    print("Healthy % by glcm prediction:")
    print(predicted_healthy_percent)
    print("Disease % by glcm prediction:")
    print(predicted_disease_percent)

    # Classification logic
    if predicted_disease_percent < 5 and mask_disease_percent < 5:
        print("Confidently Healthy")
    elif (10 > predicted_disease_percent > 5 and mask_disease_percent < 5) or (predicted_disease_percent < 5 and 10 > mask_disease_percent > 5):
        print("Probably Healthy")
    elif predicted_disease_percent > 20 and mask_disease_percent > 20:
        print("Confidently Diseased")
    else:
        print("Probably Diseased")

    return predicted_disease_percent


In [ ]:
print("Healthy % by color detection",100-disease_percentage)
print("Disease % by color detection",disease_percentage)
glcm_predicted_disease_percent = predict_disease_glcm(contrast, correlation, energy, homogeneity, asm, dissimilarity, disease_percentage)

In [ ]:
def predict_disease_glcm_mrmr(contrast, correlation, energy, homogeneity, asm, dissimilarity, mask_disease_percent, model_path="/content/drive/MyDrive/leaf project/glcm_regressor_mrmr.pkl"):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")  # Suppress all warnings
        model = joblib.load(model_path)
        features = np.array([[contrast, correlation, energy, homogeneity, asm, dissimilarity]])
        predicted_disease_percent = model.predict(features)[0]

    predicted_healthy_percent = 100 - predicted_disease_percent
    print("Healthy % by glcm prediction:")
    print(predicted_healthy_percent)
    print("Disease % by glcm prediction:")
    print(predicted_disease_percent)

    # Classification logic
    if predicted_disease_percent < 5 and mask_disease_percent < 5:
        print("Confidently Healthy")
    elif (10 > predicted_disease_percent > 5 and mask_disease_percent < 5) or (predicted_disease_percent < 5 and 10 > mask_disease_percent > 5):
        print("Probably Healthy")
    elif predicted_disease_percent > 20 and mask_disease_percent > 20:
        print("Confidently Diseased")
    else:
        print("Probably Diseased")

    return predicted_disease_percent


In [ ]:
print("GLCM mrmr")
print("Healthy % by color detection",100-disease_percentage)
print("Disease % by color detection",disease_percentage)
glcm_predicted_disease_percent = predict_disease_glcm_mrmr(contrast, correlation, energy, homogeneity, asm, dissimilarity, disease_percentage)

glcm_predicted_disease_percent -> DISEASE % BY GLCM

# **GET LEAF COLOR CLUSTERS GREEN, LIGHT, MILD, DARK AND THEIR PIXEL COUNTS**

In [ ]:
def categorize_pixels(mask_green, hsv_image, color_ranges, disease_mask):
    # Initialize counts
    categories = {
        "green": np.sum(mask_green == 255),
        "light": 0,
        "mild": 0,
        "dark": 0
    }

    # Create masks for each category safely
    skin_mask = cv2.inRange(hsv_image, *color_ranges['skin']) if 'skin' in color_ranges else np.zeros_like(mask_green)
    orange_mask = cv2.inRange(hsv_image, *color_ranges['orange']) if 'orange' in color_ranges else np.zeros_like(mask_green)
    yellow_mask = cv2.inRange(hsv_image, *color_ranges['yellow']) if 'yellow' in color_ranges else np.zeros_like(mask_green)
    brown_mask = cv2.inRange(hsv_image, *color_ranges['brown']) if 'brown' in color_ranges else np.zeros_like(mask_green)
    gray_mask = cv2.inRange(hsv_image, *color_ranges['gray']) if 'gray' in color_ranges else np.zeros_like(mask_green)
    red_mask = cv2.inRange(hsv_image, *color_ranges['red']) if 'red' in color_ranges else np.zeros_like(mask_green)


    # Combine light masks
    light_mask = cv2.bitwise_or(skin_mask, orange_mask)

    # Combine dark masks (brown, gray, red)
    dark_mask = cv2.bitwise_or(brown_mask, gray_mask)
    dark_mask = cv2.bitwise_or(dark_mask, red_mask)

    # Assign pixels to categories based on hierarchy
    disease_pixels_hsv = hsv_image[disease_mask == 255]
    dark_pixels = cv2.bitwise_and(disease_mask, dark_mask)  # Pixels classified as dark
    remaining_mask = cv2.bitwise_and(disease_mask, cv2.bitwise_not(dark_pixels))
    mild_pixels = cv2.bitwise_and(remaining_mask, yellow_mask)  # Pixels classified as mild
    remaining_mask = cv2.bitwise_and(remaining_mask, cv2.bitwise_not(mild_pixels))
    light_pixels = cv2.bitwise_and(remaining_mask, light_mask)  # Pixels classified as light

    green_pixels=mask_green

    # Print pixel counts for debugging
    print(f"Green Pixels: {np.sum(mask_green == 255)}")
    print(f"Light Pixels: {np.sum(light_pixels == 255)}")
    print(f"Mild Pixels: {np.sum(mild_pixels == 255)}")
    print(f"Dark Pixels: {np.sum(dark_pixels == 255)}")

    # Collect valid masks (those that are not empty)
    masks = [("Green", green_pixels), ("Light", light_pixels), ("Mild", mild_pixels), ("Dark", dark_pixels)]
    valid_masks = [(name, mask) for name, mask in masks if np.any(mask)]  # Keep only non-empty masks

    # Update counts
    categories["dark"] = np.sum(dark_pixels == 255)
    categories["mild"] = np.sum(mild_pixels == 255)
    categories["light"] = np.sum(light_pixels == 255)

    # Visualization using matplotlib (from HSV)
    plt.figure(figsize=(12, 6))
    for idx, (name, mask) in enumerate(masks, 1):
        colored_area_hsv = cv2.bitwise_and(hsv_image, hsv_image, mask=mask)
        colored_area_rgb = cv2.cvtColor(colored_area_hsv, cv2.COLOR_HSV2RGB)
        plt.subplot(2, 2, idx)
        plt.imshow(colored_area_rgb)
        plt.title(f"{name} Area (HSV-based)")
        plt.axis("off")

    plt.tight_layout()
    plt.show()


    return categories, green_pixels, light_pixels, mild_pixels, dark_pixels, masks



In [ ]:
# Additional Step: Analyze HSV values of diseased areas
hsv_image = cv2.cvtColor(cv2.cvtColor(np.array(output_image), cv2.COLOR_RGB2BGR), cv2.COLOR_BGR2HSV)
categories, green_pixels, light_pixels, mild_pixels, dark_pixels, masks = categorize_pixels(mask_green, hsv_image, color_ranges, mask_combined)


hsv_image -> BACKGROUND REMOVED IMG IN HSV

categories -> DICT LIKE "GREEN" : X (no of green pixels) AND SO ON

green_pixels -> ALL GREEN PIXELS

light_pixels -> ALL LIGHT PIXELS

mild_pixels -> ALL MILD PIXELS

dark_pixels -> ALL DARK PIXELS

masks -> DICT LIKE "GREEN" : green_pixels AND SO ON

# **GET LAB COLOR VALUES FOR EACH PIXEL IN EACH CLUSTER**

In [ ]:
def get_lab_colors(image, masks):

    # Convert the image to BGR (for OpenCV), then to LAB
    image_bgr = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
    image_lab = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2LAB)

    lab_values_by_category = {}

    for name, mask in masks:
        # Apply the mask to get LAB values only where mask is 255
        lab_pixels = image_lab[mask == 255]
        if lab_pixels.size > 0:
            lab_values_by_category[name.lower()] = [tuple(pixel) for pixel in lab_pixels]

    return lab_values_by_category


In [ ]:
lab_color_dict = get_lab_colors(output_image, masks)
# print("green:", lab_color_dict['green'][:3])
# print("mild:", lab_color_dict['mild'][:3])
# print("light:", lab_color_dict['light'][:3])
# print("dark:", lab_color_dict['dark'][:3])

if 'green' in lab_color_dict:
    print("green:", lab_color_dict['green'][:3])
else:
    print("green:", [])

if 'mild' in lab_color_dict:
    print("mild:", lab_color_dict['mild'][:3])
else:
    print("mild:", [])

if 'light' in lab_color_dict:
    print("light:", lab_color_dict['light'][:3])
else:
    print("light:", [])

if 'dark' in lab_color_dict:
    print("dark:", lab_color_dict['dark'][:3])
else:
    print("dark:", [])


In [ ]:
def plot_lab_distributions(lab_color_dict):

    plt.figure(figsize=(16, 10))

    for i, (category, values) in enumerate(lab_color_dict.items(), 1):
        lab_array = np.array(values)
        L_vals, A_vals, B_vals = lab_array[:, 0], lab_array[:, 1], lab_array[:, 2]

        plt.subplot(len(lab_color_dict), 3, (i - 1) * 3 + 1)
        plt.hist(L_vals, bins=50, color='gray')
        plt.title(f'{category.capitalize()} - L (Lightness)')
        plt.xlabel('L')
        plt.ylabel('Count')

        plt.subplot(len(lab_color_dict), 3, (i - 1) * 3 + 2)
        plt.hist(A_vals, bins=50, color='red')
        plt.title(f'{category.capitalize()} - A (Green-Red)')
        plt.xlabel('A')
        plt.ylabel('Count')

        plt.subplot(len(lab_color_dict), 3, (i - 1) * 3 + 3)
        plt.hist(B_vals, bins=50, color='blue')
        plt.title(f'{category.capitalize()} - B (Blue-Yellow)')
        plt.xlabel('B')
        plt.ylabel('Count')

    plt.tight_layout()
    plt.show()

plot_lab_distributions(lab_color_dict)


In [ ]:
def visualize_lab_summary(lab_color_dict):
    from collections import Counter

    summary_dict = {}

    fig, axs = plt.subplots(len(lab_color_dict), 2, figsize=(6, 3 * len(lab_color_dict)))
    if len(lab_color_dict) == 1:
        axs = [axs]  # Make axs iterable if only 1 cluster

    for idx, (key, lab_values) in enumerate(lab_color_dict.items()):
        lab_array = np.array(lab_values, dtype=np.uint8)

        # Average LAB
        avg_lab = np.mean(lab_array, axis=0).astype(np.uint8)
        avg_lab_img = avg_lab.reshape(1, 1, 3)
        avg_rgb = cv2.cvtColor(avg_lab_img, cv2.COLOR_LAB2RGB).reshape(3)

        # Most frequent LAB
        lab_tuples = [tuple(val) for val in lab_array]
        most_common_lab = Counter(lab_tuples).most_common(1)[0][0]
        most_common_lab_arr = np.array(most_common_lab, dtype=np.uint8).reshape(1, 1, 3)
        common_rgb = cv2.cvtColor(most_common_lab_arr, cv2.COLOR_LAB2RGB).reshape(3)

        # Print values
        print(f"{key} - Avg LAB: {avg_lab.tolist()}")
        print(f"{key} - Most Frequent LAB: {most_common_lab}")

        # Save to summary dict
        summary_dict[key] = (tuple(avg_lab.tolist()), most_common_lab)

        # Plotting
        axs[idx][0].imshow(np.ones((50, 100, 3), dtype=np.uint8) * avg_rgb)
        axs[idx][0].set_title(f"{key} - Avg LAB")
        axs[idx][0].axis("off")

        axs[idx][1].imshow(np.ones((50, 100, 3), dtype=np.uint8) * common_rgb)
        axs[idx][1].set_title(f"{key} - Most Frequent LAB")
        axs[idx][1].axis("off")

    plt.tight_layout()
    plt.show()

    return summary_dict


In [ ]:
summary_dict = visualize_lab_summary(lab_color_dict)

lab_color_dict -> DICT LIKE "GREEN" : [ (x,y,z) , (a,b,c) ...] AND SO ON

summary_dict -> DICT LIKE "GREEN" : [ (avg- x,y,z) , (most freq- a,b,c) ] AND SO ON

# DISTANCE FORMULAS

In [ ]:
def euclidean_distance(lab1, lab2):
    return np.sqrt(np.sum((np.array(lab1) - np.array(lab2))**2))

def manhattan_distance(lab1, lab2):
    return np.sum(np.abs(np.array(lab1) - np.array(lab2)))

def chebyshev_distance(lab1, lab2):
    return np.max(np.abs(np.array(lab1) - np.array(lab2)))

def canberra_distance(lab1, lab2):
    return np.sum(np.abs(np.array(lab1) - np.array(lab2)) / (np.array(lab1) + np.array(lab2) + 1e-10))

def minkowski_distance(lab1, lab2, p=3):
    return np.sum(np.abs(np.array(lab1) - np.array(lab2)) ** p) ** (1/p)

def mahalanobis_distance(lab1, lab2, inv_cov_matrix):
    diff = np.array(lab1) - np.array(lab2)
    return np.sqrt(np.dot(np.dot(diff.T, inv_cov_matrix), diff))

# CLOSENESS FINDING FOR EACH CLUSTER

In [ ]:
# LAB values for each cluster
lab_palette_dict= {
    "green": [
        (180, 102, 183), (172, 98, 183), (169, 98, 183), (168, 100, 183), (168, 98, 180),
        (165, 98, 183), (165, 97, 181), (164, 100, 180), (162, 98, 180), (161, 97, 182),
        (160, 98, 177), (159, 99, 182), (159, 99, 181), (152, 98, 180), (118, 111, 144)
    ],
    "light": [
        (230, 128, 140), (210, 128, 140), (200, 128, 140), (195, 130, 145), (190, 132, 150),
        (165, 130, 155), (160, 132, 160), (155, 134, 165), (150, 136, 170), (145, 138, 175),
        (140, 140, 180), (135, 142, 185), (130, 144, 190), (125, 146, 195), (120, 148, 200)
    ],
    "mild": [
        (208, 123, 189), (204, 119, 181), (202, 117, 182), (201, 123, 185), (199, 117, 180),
        (198, 119, 180), (197, 117, 184), (192, 116, 178), (191, 117, 182), (189, 119, 181),
        (187, 117, 182), (185, 115, 175), (181, 115, 174), (173, 118, 176), (166, 117, 178)
    ],
    "dark": [
        (136, 136, 154), (130, 136, 154), (120, 136, 150), (119, 130, 149), (115, 136, 147),
        (112, 132, 147), (111, 135, 147), (110, 132, 147), (106, 136, 150), (105, 137, 155),
        (104, 133, 146), (80, 130, 135), (60, 129, 136), (40, 128, 135), (20, 128, 135)
    ]
}

# Convert LAB to RGB for visualization
def lab_to_rgb(lab_list):
    rgb_list = []
    for L, A, B in lab_list:
        lab_pixel = np.uint8([[[L, A, B]]])
        rgb_pixel = cv2.cvtColor(lab_pixel, cv2.COLOR_LAB2RGB)[0][0]
        rgb_list.append(rgb_pixel)
    return rgb_list

rgb_colors_green = lab_to_rgb(lab_palette_dict["green"])
rgb_colors_light = lab_to_rgb(lab_palette_dict["light"])
rgb_colors_mild  = lab_to_rgb(lab_palette_dict["mild"])
rgb_colors_dark  = lab_to_rgb(lab_palette_dict["dark"])

# Function to display colors
def display_colors(color_list, title):
    fig, ax = plt.subplots(1, 15, figsize=(15, 2))
    fig.suptitle(title, fontsize=14)

    for i in range(15):
        ax[i].imshow([[color_list[i]]])
        ax[i].axis("off")

    plt.show()

# Show all color ranges
display_colors(rgb_colors_green, "Green Shades")
display_colors(rgb_colors_light, "Light Shades")
display_colors(rgb_colors_mild,  "Mild Shades")
display_colors(rgb_colors_dark,  "Dark Shades")


**USING EUCLIDEAN DISTANCE**

In [ ]:
# def compute_euclid_closeness_table(source_cluster_name):
#     palette_colors = lab_palette_dict[source_cluster_name]
#     table = []

#     for i, palette_lab in enumerate(palette_colors, 1):
#         row = {}
#         for target_cluster, target_vals in lab_color_dict.items():
#             dists = [euclidean_distance(palette_lab, t_lab) for t_lab in target_vals]
#             avg_dist = np.mean(dists)
#             row[target_cluster] = round(avg_dist, 2)
#         table.append(row)

#     df = pd.DataFrame(table)
#     df.index = range(1, len(df) + 1)
#     df.index.name = "#"

#     # Add row for column-wise averages
#     avg_row = df.mean().round(2)
#     avg_row.name = "Average"
#     df = pd.concat([df, pd.DataFrame([avg_row])])

#     return df


# # Generate and print tables
# green_table = compute_euclid_closeness_table("green")
# light_table = compute_euclid_closeness_table("light")
# mild_table = compute_euclid_closeness_table("mild")
# dark_table = compute_euclid_closeness_table("dark")

# print("\nCloseness Table for GREEN Cluster:")
# print(green_table.to_string())

# print("\nCloseness Table for LIGHT Cluster:")
# print(light_table.to_string())

# print("\nCloseness Table for MILD Cluster:")
# print(mild_table.to_string())

# print("\nCloseness Table for DARK Cluster:")
# print(dark_table.to_string())


**USING CHEBYSHEV DISTANCE**

In [ ]:
# def compute_cheby_closeness_table(source_cluster_name):
#     palette_colors = lab_palette_dict[source_cluster_name]
#     table = []

#     for i, palette_lab in enumerate(palette_colors, 1):
#         row = {}
#         for target_cluster, target_vals in lab_color_dict.items():
#             dists = [chebyshev_distance(palette_lab, t_lab) for t_lab in target_vals]
#             avg_dist = np.mean(dists)
#             row[target_cluster] = round(avg_dist, 2)
#         table.append(row)

#     df = pd.DataFrame(table)
#     df.index = range(1, len(df) + 1)
#     df.index.name = "#"

#     # Add row for column-wise averages
#     avg_row = df.mean().round(2)
#     avg_row.name = "Average"
#     df = pd.concat([df, pd.DataFrame([avg_row])])

#     return df

# # Generate and print tables
# green_table1 = compute_cheby_closeness_table("green")
# light_table1 = compute_cheby_closeness_table("light")
# mild_table1 = compute_cheby_closeness_table("mild")
# dark_table1 = compute_cheby_closeness_table("dark")

# print("\nCloseness Table for GREEN Cluster:")
# print(green_table1.to_string())

# print("\nCloseness Table for LIGHT Cluster:")
# print(light_table1.to_string())

# print("\nCloseness Table for MILD Cluster:")
# print(mild_table1.to_string())

# print("\nCloseness Table for DARK Cluster:")
# print(dark_table1.to_string())


lab_color_dict -> MANUAL LAB COLOR SHADES

green_table -> CLUSTER GREEN - CLOSENESS TO 4*15 COLOR SHADES

light_table -> CLUSTER LIGHT - CLOSENESS TO 4*15 COLOR SHADES

mild_table -> CLUSTER MILD - CLOSENESS TO 4*15 COLOR SHADES

dark_table -> CLUSTER DARK - CLOSENESS TO 4*15 COLOR SHADES

# FUZZY SUB CLUSTERING INTO COLOR STRIPS

In [ ]:
def fuzzy_c_subclustering(lab_color_dict):
    sub_clusters = {}
    cluster_stats = {}  # <- Added dictionary for cluster stats
    fig, axes = plt.subplots(4, 3, figsize=(12, 6))
    fig.suptitle("Cluster Color Strips with Percentages", fontsize=14)

    keys = list(lab_color_dict.keys())
    for row_idx, key in enumerate(keys):
        pixels = lab_color_dict[key]
        if len(pixels) < 3:
            for i in range(3):
                axes[row_idx, i].axis("off")
            continue

        data = np.array(pixels).T
        cntr, u, _, _, _, _, _ = fuzz.cluster.cmeans(data, 3, 2.0, error=0.005, maxiter=1000)
        labels = np.argmax(u, axis=0)

        stats = []
        sub_clusters[key] = {}
        for cluster_idx in range(3):
            cluster_pixels = [pixels[j] for j in range(len(pixels)) if labels[j] == cluster_idx]
            percent = len(cluster_pixels) / len(pixels) * 100
            avg_lab = np.mean(cluster_pixels, axis=0) if cluster_pixels else [0, 0, 0]
            stats.append({'cluster': cluster_idx, 'percent': percent, 'avg_lab': avg_lab})

            sub_clusters[key][f'cluster {cluster_idx + 1}'] = cluster_pixels

            cluster_lab = np.array(cluster_pixels, dtype=np.uint8).reshape(-1, 1, 3)
            cluster_rgb = cv2.cvtColor(cluster_lab, cv2.COLOR_Lab2RGB).reshape(-1, 3)

            strip_height = max(20, int(len(cluster_rgb) * 0.2))  # Adjust multiplier as needed
            strip = np.zeros((strip_height, len(cluster_rgb), 3), dtype=np.uint8)
            for x in range(len(cluster_rgb)):
                strip[:, x] = cluster_rgb[x]

            axes[row_idx, cluster_idx].imshow(strip)
            axes[row_idx, cluster_idx].set_title(f"{key} - {percent:.1f}%", fontsize=9)
            axes[row_idx, cluster_idx].axis("off")

        cluster_stats[key] = stats  # <- Save the stats for this key

    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()

    return sub_clusters, cluster_stats  # <- Return both

sub_clusters, cluster_stats = fuzzy_c_subclustering(lab_color_dict)
pprint.pprint(cluster_stats)


In [ ]:


def visualize_cluster_averages(cluster_stats):
    fig, axs = plt.subplots(4, 3, figsize=(12, 16))
    keys = ['dark', 'green', 'light', 'mild']

    for i, key in enumerate(keys):
        clusters = cluster_stats.get(key, [])
        for j, cluster in enumerate(clusters[:3]):
            avg_lab = cluster['avg_lab'].astype(np.uint8).reshape(1, 1, 3)
            avg_rgb = cv2.cvtColor(avg_lab, cv2.COLOR_LAB2RGB).reshape(3)

            axs[i, j].imshow(np.ones((100, 100, 3), dtype=np.uint8) * avg_rgb)
            axs[i, j].set_title(f"{key} Cluster {j }")
            axs[i, j].axis("off")

    plt.tight_layout()
    plt.show()

visualize_cluster_averages(cluster_stats)


# DISTANCE BTW AVG COLOR OF EACH STRIP AND ALL 15*4 MANUAL PALETTE COLORS

In [ ]:
def compute_cluster_distances(cluster_stats, lab_palette_dict):
    distance_table = {}

    for label_key, cluster_list in cluster_stats.items():  # e.g., 'green', 'light', etc.
        distance_table[label_key] = {}

        for cluster_info in cluster_list:  # Each cluster: {'cluster': x, 'avg_lab': ..., 'percent': ...}
            cluster_id = f"cluster {cluster_info['cluster'] + 1}"
            avg_lab = cluster_info['avg_lab']

            palette_distances = {}
            for palette_key, palette_colors in lab_palette_dict.items():  # e.g., 'green', 'brown', etc.
                distances = [euclidean_distance(avg_lab, palette_lab) for palette_lab in palette_colors]
                palette_distances[palette_key] = distances  # List of 15 distances

            distance_table[label_key][cluster_id] = palette_distances

    return distance_table
distance_table = compute_cluster_distances(cluster_stats, lab_palette_dict)


# DISTANCE SCORE AND CLUSTER INDEX

DISTANCE SCORE->

dist x 1 + dist x 2 +.....+ dist x 15 for GREEN

dist x 15 + dist x 14 +.....+ dist x 1 for OTHERS

CLUSTER INDEX->

distance score x percent/100

In [ ]:
def update_cluster_stats_with_dist_score(distance_table, cluster_stats):
    for key in distance_table:
        for cluster_name, distances in distance_table[key].items():
            weights = list(range(1, 16))

            # Convert distances to numeric if they are strings
            distances_numeric = [float(d) for d in distances['green']] # distances was previously a dictionary, this change makes it a list of numeric values

            weighted_sum = sum(weight * dist for weight, dist in zip(weights, distances_numeric)) # distances is now distances_numeric
            weight_sum = sum(weights)
            dist_score = weighted_sum / weight_sum

            cluster_id = int(cluster_name.split()[-1])
            for stat in cluster_stats[key]:
                if stat['cluster'] == cluster_id - 1:  #cluster id was off by 1
                    stat['dist_score'] = dist_score
                    stat['cluster_index'] = dist_score * (stat['percent'])/100
    return cluster_stats


update_cluster_stats_with_dist_score(distance_table, cluster_stats)

# SEVERITY SCORE BY COLOR INDEX

GREEN (HEALTHY) INDEX -> sum of green sub cluster indexes x %age of green pixels

OTHER (DISEASED) INDEXES -> similary for light,mild,dark clusters and pixels

SEVERITY SCORE -> HEALTHY INDEX - DISEASED INDEX

In [ ]:
# Total pixel count
total_pixels = sum(categories.values())

# Compute indexes using cluster_stats and categories
# green_index = sum(c['cluster_index'] for c in cluster_stats['green']) * (categories['green'] / total_pixels)
# light_index = sum(c['cluster_index'] for c in cluster_stats['light']) * (categories['light'] / total_pixels)
# mild_index  = sum(c['cluster_index'] for c in cluster_stats['mild'])  * (categories['mild']  / total_pixels)
# dark_index  = sum(c['cluster_index'] for c in cluster_stats['dark'])  * (categories['dark']  / total_pixels)

green_index = sum(c['cluster_index'] for c in cluster_stats.get('green', [])) * (categories.get('green', 0) / total_pixels) if 'green' in cluster_stats else 0
light_index = sum(c['cluster_index'] for c in cluster_stats.get('light', [])) * (categories.get('light', 0) / total_pixels) if 'light' in cluster_stats else 0
mild_index  = sum(c['cluster_index'] for c in cluster_stats.get('mild',  [])) * (categories.get('mild',  0) / total_pixels) if 'mild'  in cluster_stats else 0
dark_index  = sum(c['cluster_index'] for c in cluster_stats.get('dark',  [])) * (categories.get('dark',  0) / total_pixels) if 'dark'  in cluster_stats else 0



# Print individual indexes
print("Green Index:", green_index)
print("Light Index:", light_index)
print("Mild Index:", mild_index)
print("Dark Index:", dark_index)

# Final severity score
severity_score = ( light_index + mild_index + dark_index ) -(green_index)
# severity_score = ( light_index + 1.5*mild_index + 2*dark_index ) -(green_index)
print("Severity Score:", severity_score)

clamped_score = max(-60, min(severity_score, 60))

normalized_score = ((clamped_score + 60) / 120) * 100


# clamped_score = max(-60, min(severity_score, 100))

# normalized_score = ((clamped_score + 60) / 160) * 100


print("Normalized Severity Score (0-100):", normalized_score)

In [ ]:
def visualize_results1(input_image, output_image, green_mask, disease_mask,
                      leaf_type, disease_percentage, glcm_predicted_disease_percent,
                      severity_score, normalized_score):

    green_overlay = cv2.cvtColor(green_mask, cv2.COLOR_GRAY2BGR)
    disease_overlay = cv2.cvtColor(disease_mask, cv2.COLOR_GRAY2BGR)
    green_overlay[np.where((green_overlay == [255, 255, 255]).all(axis=2))] = [0, 255, 0]
    disease_overlay[np.where((disease_overlay == [255, 255, 255]).all(axis=2))] = [0, 0, 255]

    output_image_bgr = cv2.cvtColor(np.array(output_image), cv2.COLOR_RGB2BGR)
    combined_overlay = cv2.addWeighted(output_image_bgr, 1.0, disease_overlay, 0.3, 0)
    combined_overlay_rgb = cv2.cvtColor(combined_overlay, cv2.COLOR_BGR2RGB)

    fig, axes = plt.subplots(1, 3, figsize=(14, 6))
    axes[0].imshow(input_image)
    axes[0].set_title("Original Image")
    axes[1].imshow(output_image)
    axes[1].set_title("Background Removed")
    axes[2].imshow(combined_overlay_rgb)
    axes[2].set_title("Disease Overlay")
    for ax in axes:
        ax.axis("off")


    full_text = (f"Leaf: {leaf_type},Disease:  {disease}, Disease Pixel %: {disease_percentage:.2f}%\n"
                 f"GLCM Predicted Disease %: {glcm_predicted_disease_percent:.2f}%\n"
                 f"Severity Score: {severity_score:.2f}, Severity%: {normalized_score:.2f}")

    fig.text(0.5, 0.02, full_text, ha="center", fontsize=11, color="red",
             bbox={"facecolor": "white", "alpha": 0.8, "pad": 5})

    plt.tight_layout(rect=[0, 0.07, 1, 1])
    plt.show()



visualize_results1(input_image, output_image, mask_green, mask_combined,
                  leaf_type, disease_percentage,
                  glcm_predicted_disease_percent, severity_score, normalized_score)
